In [1]:
import os
os.chdir("../")

In [36]:
import matplotlib.pyplot as plt

from hyperparameterStudy.mil_postprocessing import MilPostprocessor
from PreprocessMILImageData import PreprocessMILImageData
from PreprocessData import PreprocessData
from PreprocessImageData import PreprocessImageData
import tensorflow.keras as k
import tensorflow as tf
import matplotlib.pyplot as plt

data_type = "test"
model_name = "ave_pool_selu_lay4_little_drop_little_l2_global_ave_pooling_n32_zeros_afalse_no_noise_fft_denoise5_residual_supervised_cfalse_nfalse_images_lfalse"
model_path = "results/hyperparameter_study/supervised/models"
file_list = PreprocessData.load_file_list(data_type, angio_or_structure="images")
pid = PreprocessImageData(input_file_list=file_list, rgb=False, crop=False, data_type=data_type)
pid.preprocess_data_and_save()
ds = pid.create_dataset_for_calculation()

# visualize_file_list = PreprocessMILImageData.load_file_list("test", angio_or_structure="images")
# visualize_file_list = sorted(visualize_file_list, key=lambda file: (int(file[1]),
#                                                                     int(file[0].split("_")[-1][:-4])))
pretrained_model = k.models.load_model(os.path.join(model_path, model_name))
ds_norm = ds.batch(1)
#image_probs = pretrained_model.predict(ds_norm, verbose=1)

loss_object = k.losses.BinaryCrossentropy()


# for elem in ds.skip(4).take(1):
#   plt.figure()
#   plt.imshow(elem[0], "gray")
#   plt.show()

#ds = ds.batch(1).map(create_adversarial_pattern)
# ds_adv = ds.batch(1).map(create_adversarial_pattern)
# image_probs_adv = pretrained_model.predict(ds_adv, verbose=1)


In [3]:
def create_adversarial_pattern(input_image, input_label):
    with tf.GradientTape() as tape:
      tape.watch(input_image)
      prediction = pretrained_model(input_image)
      loss = loss_object(input_label, prediction)

    # Get the gradients of the loss w.r.t to the input image.
    gradient = tape.gradient(loss, input_image)
    # Get the sign of the gradients to create the perturbation
    signed_grad = tf.sign(gradient)
    #multiplier = input_image.max() - gradient.max()
    eps = 0.1
    adv_image = input_image + eps*signed_grad
    return adv_image, input_label

In [49]:
image_probs = pretrained_model.predict(ds_norm, verbose=1)

ds_adv = ds.batch(1).map(create_adversarial_pattern)
image_probs_adv = pretrained_model.predict(ds_adv, verbose=1)

30/30 [==============================] - 847s 28s/step


In [50]:
label_list = []
for elem in ds_norm:
    label_list.append(elem[1].numpy())

In [51]:
from sklearn import metrics
auc = metrics.roc_auc_score(label_list, image_probs)
print(auc)

0.9261363636363636


In [52]:
from sklearn import metrics
auc = metrics.roc_auc_score(label_list, image_probs_adv)
print(auc)

0.5284090909090909


In [53]:
image_probs_adv == image_probs

array([[ True],
       [ True],
       [ True],
       [False],
       [False],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [False],
       [ True],
       [ True],
       [False],
       [False],
       [ True],
       [ True],
       [False],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [False],
       [False],
       [False],
       [ True],
       [ True],
       [False],
       [ True]])

In [7]:
data_type = "test"
model_name = "ave_pool_selu_lay5_no_drop_little_l2_global_ave_pooling_n32_zeros_afalse_no_noise_fft_denoise5_residual_mil_cfalse_nfalse_images_lfalse"
model_path = "results/hyperparameter_study/mil/models"
file_list = PreprocessData.load_file_list(data_type, angio_or_structure="images")
pid = PreprocessMILImageData(input_file_list=file_list, rgb=False, crop=False, data_type=data_type)
pid.preprocess_data_and_save()
mil_ds = pid.create_dataset_for_calculation()
pretrained_model = k.models.load_model(os.path.join(model_path, model_name))
mil_ds_norm = mil_ds.batch(1)

mil_probs = pretrained_model.predict(mil_ds_norm, verbose=1)

mil_ds_adv = mil_ds.batch(1).map(create_adversarial_pattern)
mil_probs_adv = pretrained_model.predict(mil_ds_adv, verbose=1)

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 3080 Ti, compute capability 8.6


2022-07-19 00:19:34.780236: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


3000/3000 [==============================] - 100290s 33s/step


In [59]:
pid = PreprocessMILImageData(input_file_list=file_list, rgb=False, crop=False, data_type=data_type)
pid.preprocess_data_and_save()
mil_ds = pid.create_dataset_for_calculation()

In [60]:
mil_label_list = [label for file, label in pid.calculation_file_list]
mil_image_label_list = mil_label_list[5::100]
print(len(mil_image_label_list))
print(mil_image_label_list)

30
[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [61]:
from mil_pooling import MilPooling
mp = MilPooling(mil_probs)
bag_props = mp.weighted_average()

In [62]:
mp = MilPooling(mil_probs_adv)
bag_props_adv = mp.weighted_average()

In [63]:
print(bag_props)
print(bag_props_adv)

[-233.2  -214.2  -236.9  -191.5  -232.5  -271.   -297.8  -150.5   696.5
 -159.8    -8.42 -217.8   237.    135.9  -150.6   297.2   186.5   385.8
  407.5   465.5   320.2   309.5  -184.1  -120.3  -138.2  -132.6  -178.6
  -21.   -139.6  -163.1 ]
[-202.4 -182.  -204.6 -148.4 -200.2 -238.9 -265.5 -116.1  729.  -169.2
  -76.1 -190.4  226.1   65.1 -148.   313.8  135.6  383.   408.   478.8
  301.2  277.  -151.9 -136.1 -167.9 -144.6 -157.9 -128.1 -123.4 -166.1]


In [64]:
auc = metrics.roc_auc_score(mil_label_list, mil_probs)
print("MIL AUC")
print(auc)

MIL AUC
0.8417576704545455


In [65]:
auc = metrics.roc_auc_score(mil_label_list, mil_probs_adv)
print("MIL AUC ADV")
print(auc)

MIL AUC ADV
0.7614272727272727


In [66]:
auc = metrics.roc_auc_score(mil_image_label_list, bag_props)
print("BAG AUC")
print(auc)

BAG AUC
0.9545454545454546


In [67]:
auc = metrics.roc_auc_score(mil_image_label_list, bag_props)
print("BAG AUC ADV")
print(auc)

BAG AUC ADV
0.9545454545454546


In [11]:
mil_probs_adv

array([[-2.389],
       [-2.236],
       [-1.136],
       ...,
       [-1.267],
       [-1.239],
       [-1.399]], dtype=float16)

In [13]:
mil_probs[2864]

array([0.2676], dtype=float16)

In [10]:
mil_probs == mil_probs_adv

array([[ True],
       [ True],
       [ True],
       ...,
       [ True],
       [ True],
       [ True]])

In [ ]:
for elem in ds.batch(1).skip(4).take(1):
  print(elem[0].shape)
  plt.figure(figsize=(12, 12))
  plt.imshow(elem[0][0, :, :, 0], "gray")
  plt.colorbar()

In [ ]:
input_image = elem[0]
input_label = elem[1]
with tf.GradientTape() as tape:
  tape.watch(input_image)
  prediction = pretrained_model(input_image)
  loss = loss_object(input_label, prediction)
  # Get the gradients of the loss w.r.t to the input image.
  gradient = tape.gradient(loss, input_image)


In [ ]:
for elem in ds.batch(1).skip(4).take(1):
  print(elem[0].shape)
  plt.figure(figsize=(12, 12))
  plt.imshow((elem[0] + gradient)[0, :, :, 0] , "gray")
  plt.colorbar()

In [ ]:
ins, adv, norm = 0, 0, 0
for elem in zip(ds_norm, image_probs_adv, image_probs):
  ins += 1
  adv += (image_probs_adv > 0 and ds_norm[1] == 1.) or (image_probs_adv < 0 and ds_norm[1] == 0.)
  norm += (image_probs > 0 and ds_norm[1] == 1.) or (image_probs < 0 and ds_norm[1] == 0.)

print(adv/ins, norm/ins)

# for elem in ds.skip(4).take(1):
#   plt.figure()
#   plt.imshow(elem[0][0], "gray")
#   plt.show()


# hp = MilPostprocessor(model_name, ds, visualize_file_list, crop=False)
# hp.mil_postprocessing()